In [1]:
import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.5.0"  # Change this path according to your R installation.

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates

import seaborn as sns
import numpy as np
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import IntVector

# Enable conversion between pandas and R
pandas2ri.activate()

# Load the R base package
base = importr('base')
# Load the package depmixS4
depmixS4 = importr("depmixS4")
# Set the seed in R for reproducibility
r('set.seed(2030)')
# Set seed in Python for reproducibility
np.random.seed(2030)

In [3]:
from pymer4.models import Lmer

In [4]:
# Instalar lmerTest en R 
!R -e "install.packages('lmerTest', repos='http://cran.rstudio.com/')"


R version 4.5.0 (2025-04-11 ucrt) -- "How About a Twenty-Six"
Copyright (C) 2025 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64

R es un software libre y viene sin GARANTIA ALGUNA.
Usted puede redistribuirlo bajo ciertas circunstancias.
Escriba 'license()' o 'licence()' para detalles de distribucion.

R es un proyecto colaborativo con muchos contribuyentes.
Escriba 'contributors()' para obtener más información y
'citation()' para saber cómo citar R o paquetes de R en publicaciones.

Escriba 'demo()' para demostraciones, 'help()' para el sistema on-line de ayuda,
o 'help.start()' para abrir el sistema de ayuda HTML con su navegador.
Escriba 'q()' para salir de R.

> install.packages('lmerTest', repos='http://cran.rstudio.com/')
package 'numDeriv' successfully unpacked and MD5 sums checked
package 'lmerTest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ferna\AppData\Local\Temp\Rtmp8YFAgx\downloaded_packages
> 


Installing package into 'C:/Users/ferna/AppData/Local/R/win-library/4.5'
(as 'lib' is unspecified)
also installing the dependency 'numDeriv'

probando la URL 'http://cran.rstudio.com/bin/windows/contrib/4.5/numDeriv_2016.8-1.1.zip'
probando la URL 'http://cran.rstudio.com/bin/windows/contrib/4.5/lmerTest_3.1-3.zip'


In [5]:
# Verificar e importar lmerTest
with localconverter(default_converter + pandas2ri.converter):
    try:
        lmerTest = importr('lmerTest')
        print("El paquete lmerTest está instalado y cargado correctamente.")
    except Exception as e:
        print("Error al cargar lmerTest:", e)

NameError: name 'localconverter' is not defined